In [0]:
# all imports
import os
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from numpy import expand_dims
import numpy as np
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from PIL import Image
from keras.models import load_model
from keras.models import model_from_json
import pickle
import math
import sklearn
import cv2
from google.colab.patches import cv2_imshow
import time

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
yolo_model_path = '/content/yolo_model.h5'
sec_last_json_path = '/content/drive/My Drive/hybrid/secondLastModel.json'
sec_last_weights_path = '/content/drive/My Drive/hybrid/secondLastModel.h5'
last_json_path = '/content/drive/My Drive/hybrid/lastModel.json'
last_weights_path = '/content/drive/My Drive/hybrid/lastModel.h5'
log_reg_path = '/content/drive/My Drive/hybrid/logRModel.sav'
input_dir = '/content/input'
input_w, input_h = 416, 416

In [0]:
# !cp '/content/drive/My Drive/temp/VGG/logRModel.sav' '/content/'

In [0]:
# import pickle
# clf = pickle.load(open('/content/logRModel.sav', 'rb'))

In [0]:
def load_models():
    yolo_model = load_model(yolo_model_path)
    
    sec_last_json_file = open(sec_last_json_path, 'r')
    sec_last_model_json = sec_last_json_file.read()
    sec_last_json_file.close()
    secondLastModel = model_from_json(sec_last_model_json)
    secondLastModel.load_weights(sec_last_weights_path)
    secondLastModel.summary()
    
    last_json_file = open(last_json_path, 'r')
    last_model_json = last_json_file.read()
    last_json_file.close()
    lastModel = model_from_json(last_model_json)
    lastModel.load_weights(last_weights_path)
    lastModel.summary()
    
    clf = pickle.load(open(log_reg_path, 'rb'))
    
    return (yolo_model, secondLastModel, lastModel, clf)

In [0]:
def load_image_pixels(filename, shape):
    # load the image to get its shape
    image = load_img(filename)
    width, height = image.size
    # load the image with the required size
    image = load_img(filename, target_size=shape)
    # convert to numpy array
    image = img_to_array(image)
    # scale pixel values to [0, 1]
    image = image.astype('float32')
    image /= 255.0
    # add a dimension so that we have one sample
    image = expand_dims(image, 0)
    return image, width, height

In [0]:
class BoundBox:
	def __init__(self, xmin, ymin, xmax, ymax, objness = None, classes = None):
		self.xmin = xmin
		self.ymin = ymin
		self.xmax = xmax
		self.ymax = ymax
		self.objness = objness
		self.classes = classes
		self.label = -1
		self.score = -1
 
	def get_label(self):
		if self.label == -1:
			self.label = np.argmax(self.classes)
 
		return self.label
 
	def get_score(self):
		if self.score == -1:
			self.score = self.classes[self.get_label()]
 
		return self.score

In [0]:
def _sigmoid(x):
	return 1. / (1. + np.exp(-x))

In [0]:
def decode_netout(netout, anchors, obj_thresh, net_h, net_w):
	grid_h, grid_w = netout.shape[:2]
	nb_box = 3
	netout = netout.reshape((grid_h, grid_w, nb_box, -1))
	nb_class = netout.shape[-1] - 5
	boxes = []
	netout[..., :2]  = _sigmoid(netout[..., :2])
	netout[..., 4:]  = _sigmoid(netout[..., 4:])
	netout[..., 5:]  = netout[..., 4][..., np.newaxis] * netout[..., 5:]
	netout[..., 5:] *= netout[..., 5:] > obj_thresh
 
	for i in range(grid_h*grid_w):
		row = i / grid_w
		col = i % grid_w
		for b in range(nb_box):
			# 4th element is objectness score
			objectness = netout[int(row)][int(col)][b][4]
			if(objectness.all() <= obj_thresh): continue
			# first 4 elements are x, y, w, and h
			x, y, w, h = netout[int(row)][int(col)][b][:4]
			x = (col + x) / grid_w # center position, unit: image width
			y = (row + y) / grid_h # center position, unit: image height
			w = anchors[2 * b + 0] * np.exp(w) / net_w # unit: image width
			h = anchors[2 * b + 1] * np.exp(h) / net_h # unit: image height
			# last elements are class probabilities
			classes = netout[int(row)][col][b][5:]
			box = BoundBox(x-w/2, y-h/2, x+w/2, y+h/2, objectness, classes)
			boxes.append(box)
	return boxes

In [0]:
def correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w):
	new_w, new_h = net_w, net_h
	for i in range(len(boxes)):
		x_offset, x_scale = (net_w - new_w)/2./net_w, float(new_w)/net_w
		y_offset, y_scale = (net_h - new_h)/2./net_h, float(new_h)/net_h
		boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
		boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
		boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
		boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)

In [0]:
def do_nms(boxes, nms_thresh):
	if len(boxes) > 0:
		nb_class = len(boxes[0].classes)
	else:
		return
	for c in range(nb_class):
		sorted_indices = np.argsort([-box.classes[c] for box in boxes])
		for i in range(len(sorted_indices)):
			index_i = sorted_indices[i]
			if boxes[index_i].classes[c] == 0: continue
			for j in range(i+1, len(sorted_indices)):
				index_j = sorted_indices[j]
				if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
					boxes[index_j].classes[c] = 0

In [0]:
def bbox_iou(box1, box2):
	intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
	intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])
	intersect = intersect_w * intersect_h
	w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
	w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
	union = w1*h1 + w2*h2 - intersect
	return float(intersect) / union

In [0]:
def _interval_overlap(interval_a, interval_b):
	x1, x2 = interval_a
	x3, x4 = interval_b
	if x3 < x1:
		if x4 < x1:
			return 0
		else:
			return min(x2,x4) - x1
	else:
		if x2 < x3:
			 return 0
		else:
			return min(x2,x4) - x3

In [0]:
def get_boxes(boxes, labels, thresh):
	v_boxes, v_labels, v_scores = list(), list(), list()
	# enumerate all boxes
	for box in boxes:
		# enumerate all possible labels
		for i in range(len(labels)):
			# check if the threshold for this label is high enough
			if box.classes[i] > thresh:
				v_boxes.append(box)
				v_labels.append(labels[i])
				v_scores.append(box.classes[i]*100)
				# don't break, many labels may trigger for one box
	return v_boxes, v_labels, v_scores

In [67]:
(yolo_model, second_last_model, last_model, log_reg_model) = load_models()

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [99]:
if __name__ == '__main__':
    anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]
    class_threshold = 0.6
    labels = ['person']
    start_time = int(round(time.time() * 1000))
    for ip_img in os.listdir(input_dir):
        img_file_path = os.path.join(input_dir, ip_img)        
        image, image_w, image_h = load_image_pixels(img_file_path, (input_w, input_h))
        yhat = yolo_model.predict(image)
        image_boxes = list()
        for i in range(len(yhat)):
            image_boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, input_h, input_w)
        correct_yolo_boxes(image_boxes, image_h, image_w, input_h, input_w)
        do_nms(image_boxes, 0.5)
        v_boxes, v_labels, v_scores = get_boxes(image_boxes, labels, class_threshold)
        for i in range(len(v_boxes)):
            orig_img = cv2.imread(img_file_path)
            cropped_img = orig_img[v_boxes[i].xmin:v_boxes[i].xmax, v_boxes[i].ymin:v_boxes[i].ymax]
        imResize = cv2.resize(cropped_img, (224,224), interpolation = cv2.INTER_AREA)
        im_resize_img = np.array([imResize])
        dtPredict = second_last_model.predict(im_resize_img)

        log_reg_predict = int(log_reg_model.predict(dtPredict)[0])
        if(log_reg_predict == 1):            
            last_predict = last_model.predict(dtPredict)
            predLabs = np.apply_along_axis(np.argmax,1, last_predict)            
    end_time = int(round(time.time() * 1000))
    print((end_time - start_time)/1000)

9.032
